# Clinical NLP triage (rules-first POC)
Carga léxico, puntúa notas y exporta `outputs/predictions.csv`.

In [ ]:
import pandas as pd
lex = pd.read_csv('data/lexicon_redflags.csv'); notes = pd.read_csv('data/notes_synthetic.csv')
lex.head(), notes.head()

In [ ]:
import re
def tokenize(text): return re.findall(r"\w+[áéíóúüñÁÉÍÓÚÜÑ-]*", text.lower())
def negated(text, term, window=4):
    t = tokenize(text); tt = tokenize(term)
    for i in range(len(t)-len(tt)+1):
        if t[i:i+len(tt)] == tt:
            if any(x in ["no","niega","sin"] for x in t[max(0,i-window):i]): return True
    return False

In [ ]:
rows = []
for _, r in notes.iterrows():
    s=0; hits=[]
    for _, w in lex.iterrows():
        term=str(w["termino"]).lower()
        if term in r["triage_note"].lower() and not negated(r["triage_note"], term):
            s += int(w["peso"]); hits.append(term)
    band = "LOW" if s<3 else ("MODERATE" if s<7 else "HIGH")
    rows.append({"id": r["id"], "chief_complaint": r["chief_complaint"], "score": s, "risk_band": band, "hits": ";".join(sorted(set(hits))) })
pred = pd.DataFrame(rows).sort_values(["risk_band","score"], ascending=[False, False])
pred

In [ ]:
import os
os.makedirs('outputs', exist_ok=True)
pred.to_csv('outputs/predictions.csv', index=False, encoding='utf-8')
print('Saved outputs/predictions.csv')